In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    }
  },
  "cells": [
    {
      "cell_type": "code",
      "metadata": {
        "id": "oS98J5s0FaOc"
      },
      "source": [
        "\n",
        "import imutils #redimencionamento, rotacao\n",
        "import numpy as np\n",
        "import cv2 #import OpenCV\n",
        "from google.colab.patches import cv2_imshow\n",
        "from IPython.display import display, Javascript #webcan leitura\n",
        "from google.colab.output import eval_js #webcan leitura\n",
        "from base64 import b64decode #biblioteca para codificar dados binários\n",
        "\n",
        "def take_photo(filename='photo.jpg', quality=0.8):\n",
        "  js = Javascript('''\n",
        "    async function takePhoto(quality) {\n",
        "      const div = document.createElement('div');\n",
        "      const capture = document.createElement('button');\n",
        "      capture.textContent = 'Capture';\n",
        "      div.appendChild(capture);\n",
        "\n",
        "      const video = document.createElement('video');\n",
        "      video.style.display = 'block';\n",
        "      const stream = await navigator.mediaDevices.getUserMedia({video: true});\n",
        "\n",
        "      document.body.appendChild(div);\n",
        "      div.appendChild(video);\n",
        "      video.srcObject = stream;\n",
        "      await video.play();\n",
        "\n",
        "      // Resize the output to fit the video element.\n",
        "      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);\n",
        "\n",
        "      // Wait for Capture to be clicked.\n",
        "      await new Promise((resolve) => capture.onclick = resolve);\n",
        "\n",
        "      const canvas = document.createElement('canvas');\n",
        "      canvas.width = video.videoWidth;\n",
        "      canvas.height = video.videoHeight;\n",
        "      canvas.getContext('2d').drawImage(video, 0, 0);\n",
        "      stream.getVideoTracks()[0].stop();\n",
        "      div.remove();\n",
        "      return canvas.toDataURL('image/jpeg', quality);\n",
        "    }\n",
        "    ''')\n",
        "  display(js)\n",
        "  data = eval_js('takePhoto({})'.format(quality))\n",
        "  binary = b64decode(data.split(',')[1])\n",
        "  with open(filename, 'wb') as f:\n",
        "    f.write(binary)\n",
        "  return filename\n",
        "\n",
        "image_file = take_photo()\n",
        "\n",
        "image = cv2.imread(image_file)\n",
        "\n",
        "# redimensiona para ter uma largura máxima de 400 pixels\n",
        "image = imutils.resize(image, width=400)\n",
        "(h, w) = image.shape[:2]\n",
        "print(w,h)\n",
        "cv2_imshow(image)\n",
        "\n",
        "!wget -N https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt\n",
        "!wget -N https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel\n",
        "\n",
        "print(\"[INFO] loading model...\")\n",
        "prototxt = 'deploy.prototxt'\n",
        "model = 'res10_300x300_ssd_iter_140000.caffemodel'\n",
        "net = cv2.dnn.readNetFromCaffe(prototxt, model)\n",
        "\n",
        "# redimensiona para ter uma largura máxima de 400 pixels\n",
        "image = imutils.resize(image, width=400)\n",
        "blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))\n",
        "\n",
        "print(\"[INFO] computing object detections...\")\n",
        "net.setInput(blob)\n",
        "detections = net.forward()\n",
        "\n",
        "for i in range(0, detections.shape[2]):\n",
        "\t# extrair a probabilidade associada à previsão\n",
        "\tconfidence = detections[0, 0, i, 2]\n",
        "\n",
        "\t# filtra detecções fracas garantindo que a \"confiança\" seja\n",
        "\t# maior que o limite mínimo de confiança\n",
        "\tif confidence > 0.5: #Nossa detecção deve ter no mínimo 50% de certeza\n",
        "\t\t# calcula as coordenadas (x, y) da caixa delimitadora do objeto\n",
        "\t\tbox = detections[0, 0, i, 3:7] * np.array([w, h, w, h])\n",
        "\t\t(startX, startY, endX, endY) = box.astype(\"int\")\n",
        "\t\t# desenha a caixa delimitadora da face junto com a probabilidade associada\n",
        "\t\ttext = \"{:.2f}%\".format(confidence * 100)\n",
        "\t\ty = startY - 10 if startY - 10 > 10 else startY + 10\n",
        "\t\tcv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)\n",
        "\t\tcv2.putText(image, text, (startX, y),\n",
        "\t\t  cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)\n",
        "  \n",
        "cv2_imshow(image)"
      ],
      "execution_count": 1,
      "outputs": []
    }
  ]
}